### MID 1
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### Oversampling and smote methods used to increase size of training data.
### LSTM model used, model type is 1 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
import tensorflow as tf
import csv

In [76]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

ant_1_3_scaled = combined_data[combined_data['version'] == 1.3] 
ant_1_4_scaled = combined_data[combined_data['version'] == 1.4] 

In [4]:
# Getting training and test data.
X_train = ant_1_3_scaled[cols_to_norm]
X_train = np.array(X_train)
Y_train = ant_1_3['bug']

X_test = ant_1_4_scaled[cols_to_norm]
X_test = np.array(X_test)
Y_test = ant_1_4['bug']

In [5]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [6]:
train_x 

array([[[0.14285714, 0.6       , 0.05      , ..., 0.16595094,
         0.08571429, 0.20908494]],

       [[0.18181818, 0.        , 0.025     , ..., 0.08097907,
         0.17142857, 0.26990307]],

       [[0.03896104, 0.2       , 0.        , ..., 0.08326661,
         0.02857143, 0.1095285 ]],

       ...,

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.07792208, 0.4       , 0.        , ..., 0.15132106,
         0.05714286, 0.16428454]]])

In [7]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [8]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [9]:
test_x

array([[[0.18181818, 0.4       , 0.        , ..., 0.12421366,
         0.11428571, 0.25815673]],

       [[0.06493506, 0.4       , 0.        , ..., 0.25748599,
         0.02857143, 0.13142763]],

       [[0.16883117, 0.        , 0.025     , ..., 0.03473548,
         0.02857143, 0.12636767]],

       ...,

       [[0.2987013 , 0.        , 0.        , ..., 0.2197236 ,
         0.68571429, 0.37856087]],

       [[0.14285714, 0.        , 0.2       , ..., 0.37338962,
         0.62857143, 0.76168885]],

       [[0.05194805, 0.2       , 0.        , ..., 0.06485188,
         0.14285714, 0.28749795]]])

In [10]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [11]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [12]:
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(LSTM(80, dropout = 0.2, return_sequences=True))
model.add(LSTM(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 100)            48400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 80)             57920     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60)                33840     
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 140,221
Trainable params: 140,221
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 4ms/step - loss: 3.4786 - mse: 3.4786 - mae: 1.4965 - root_mean_squared_error: 1.8651 - mean_squared_logarithmic_error: 0.8912
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 3.4041 - mse: 3.4041 - mae: 1.4840 - root_mean_squared_error: 1.8450 - mean_squared_logarithmic_error: 0.8530
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 3.3003 - mse: 3.3003 - mae: 1.4656 - root_mean_squared_error: 1.8167 - mean_squared_logarithmic_error: 0.8026
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 3.1549 - mse: 3.1549 - mae: 1.4394 - root_mean_squared_error: 1.7762 - mean_squared_logarithmic_error: 0.7369
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.9651 - mse: 2.9651 - mae: 1.4034 - root_mean_squared_error: 1.7219 - mean_squared_logarithmic_error: 0.6579
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.6651 - mse: 2.6651 - m

4/4 [==============================] - 0s 4ms/step - loss: 1.0462 - mse: 1.0462 - mae: 0.8814 - root_mean_squared_error: 1.0229 - mean_squared_logarithmic_error: 0.2359
Epoch 47/100
4/4 [==============================] - 0s 3ms/step - loss: 0.9998 - mse: 0.9998 - mae: 0.8692 - root_mean_squared_error: 0.9999 - mean_squared_logarithmic_error: 0.2271
Epoch 48/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0060 - mse: 1.0060 - mae: 0.8694 - root_mean_squared_error: 1.0030 - mean_squared_logarithmic_error: 0.2291
Epoch 49/100
4/4 [==============================] - 0s 3ms/step - loss: 0.9147 - mse: 0.9147 - mae: 0.8317 - root_mean_squared_error: 0.9564 - mean_squared_logarithmic_error: 0.2139
Epoch 50/100
4/4 [==============================] - 0s 3ms/step - loss: 1.0461 - mse: 1.0461 - mae: 0.8834 - root_mean_squared_error: 1.0228 - mean_squared_logarithmic_error: 0.2309
Epoch 51/100
4/4 [==============================] - 0s 3ms/step - loss: 0.9688 - mse: 0.9688 - mae: 0.8

In [15]:
# Getting the predictions.
predictions_y = model.predict(test_x)
predictions_y

array([[1.36469   ],
       [1.8954377 ],
       [0.96784693],
       [1.7399871 ],
       [1.824862  ],
       [1.5361497 ],
       [1.2400578 ],
       [0.01020331],
       [0.99321526],
       [2.0420282 ],
       [1.4222519 ],
       [2.2585347 ],
       [1.0876822 ],
       [1.3847476 ],
       [1.7614616 ],
       [0.39533663],
       [2.205994  ],
       [0.6751531 ],
       [0.2871837 ],
       [1.0671968 ],
       [1.2301543 ],
       [0.78297335],
       [0.98001105],
       [1.5546768 ],
       [1.829542  ],
       [1.5134819 ],
       [1.3980005 ],
       [1.0140859 ],
       [1.4698241 ],
       [0.03218498],
       [0.        ],
       [0.62057525],
       [0.55270606],
       [1.4737132 ],
       [0.7245477 ],
       [0.17742874],
       [1.2143254 ],
       [0.10347813],
       [1.0687355 ],
       [1.0509539 ],
       [1.3815622 ],
       [0.        ],
       [0.03810829],
       [2.2186868 ],
       [0.13993487],
       [0.45986843],
       [1.7394943 ],
       [1.752

In [16]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[1.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [0.],
       [1.],
       [2.],
       [1.],
       [2.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [2.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [1.],
       [0.],
       [2.],
       [0.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],

In [17]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [18]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([197.], dtype=float32)

In [19]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.4891348], dtype=float32)

In [39]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.49753347], dtype=float32)

In [82]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 1ms/step - loss: 1.3541 - mse: 1.3541 - mae: 0.9932 - root_mean_squared_error: 1.1637 - mean_squared_logarithmic_error: 0.4857
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [22]:
# Saving the model
model_id = 1
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model1/ant1.3_ant1.4_model1_1'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model1/ant1.3_ant1.4_model1_1/assets


In [83]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [84]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [85]:
score

[1,
 'ant-1.3',
 'ant-1.4',
 1.3540980815887451,
 1.3540980815887451,
 0.9932464361190796,
 1.1636571884155273,
 0.4857074022293091,
 0.4891347885131836,
 0.4975334703922272]

In [89]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [23]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)